# Lab 2b: Enhanced MD Simulations

In this lab, we will use enhanced MD simulation techniques to accelerate sampling of an energy landscape with several local minima.

First we will load several modules from the ensembler library

In [ ]:
import tempfile
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

#Ensembler
##COde
from ensembler.potentials import OneD as potentials1D
from ensembler.potentials import TwoD as potentials2D

from ensembler.samplers.stochastic import langevinIntegrator, langevinVelocityIntegrator
from ensembler.samplers.optimizers import conjugate_gradient
from ensembler.conditions.box_conditions import periodicBoundaryCondition
from ensembler.ensemble import replica_exchange

from ensembler.system import system

##Visualisation
import ensembler.visualisation.plotPotentials as vis
from ensembler.visualisation.plotPotentials import plot_1DPotential
from ensembler.visualisation.plotSimulations import simulation_analysis_plot
from ensembler.visualisation.animationSimulation import animation_trajectory

number_of_points = 1000

## Potential defintion
We will use a four-well potential describing four local minima (=states/conformations) along a 1D (reaction) coordinate 

In [ ]:
# accessible space
space_range = [0, 10]
positions = np.linspace(space_range[0], space_range[1], number_of_points)

# build potential
V = potentials1D.fourWellPotential(
    Vmax=4, a=1.0, b=4.0, c=7.0, d=9.0,  ah=2., bh=0., ch=0.7, dh=1.3)

fig, axes = vis.plot_potential(V, positions=positions)
# fig.savefig("four_well.png")

________________________________________

## Unbiased simulation

We will perform MD simulations using the Langevin integrator to model the particle motion in a solvent. The Langevin integrator assumes that the system is coupled to a heat-bath. The interaction between the system and the heat-bath is modeled by a stochastic force term. 

The user has to set the step size $dt$ and the friction coefficient $gamma$. The initial velocity will be randomly drawn from a Maxwell-Boltzmann distribution. The temperature of the simulation and starting position will be set during the system setup (see below).

In [ ]:
# Simple Langevin integration simulation:
# Thermostat is already included (Langevin thermostat)
sim_steps = 2000
time_step = 0.1
start_position = 4
temperature = 1.0
space_range = [0, 10]

# integrator
sampler = langevinVelocityIntegrator(
    dt=time_step, gamma=20, old_position=start_position)

We are now ready to perform the simulations. The system class wraps the integrator and the potential. Additionally, the initial position of the particle $position$ as well as the temperature parameter $temperature$ are set.

To start the simulation we define the number of steps and run `sys.simulate()`. The progress of the simulation is displayed by a progress bar.

In [ ]:
# Simulation Setup
sys = system(potential=V, sampler=sampler,
             start_position=start_position,  temperature=temperature)

# simulate
cur_state = sys.simulate(sim_steps, withdraw_traj=True, init_system=False)

After running the simulation, the simulation data can be displayed as a table using `sys.trajectory`. 

In [ ]:
print("Trajectory length: ", len(sys.trajectory))
print()
print("last_state: ", cur_state)
print(len(sys.trajectory))
sys.trajectory.head(20)

In [ ]:
# visualize
simulation_analysis_plot(sys, title="Langevin Simulation",
                         limits_coordinate_space=(0, 10))

________________________________________________

### Time-independent bias
#### Umbrella sampling
Umbrella sampling is a time-independent enhanced sampling method. In this method energy barriers are overcome by adding a bias potential. Note that umbrella sampling requires the choice of a reaction coordinate along which the bias is added. The choice of a suitable reaction coordinate is non-trivial for high dimensional systems. For our low dimensional 1D we can simply chose the x-axis.

One of the most frequent umbrella sampling method uses hormonic potentials to restrain the sampling to a certain region of the potential. This is especially useful for sampling transition regions.
We define a bias potential as a hormonic oscillator centered at $x_{shift}$ and force constant $k$. In this case we want to sample the transition region around $x$ = 5.5. Therefore, we set the $x_{shift}$ parameter to 5.5. The force constant $k$ defines how much we constrain the system. The higher the energy barrier, the more constrain is needed. 

To sample the full potential energy landscape, we can set up multiple simulations with different $x_{shift}$ parameters. For subsequent analysis of umbrella sampling simulations (e.g. using the weighted histogram analysis method WHAM) it is important that the sampling space of the different simulations overlap. The higher the force constant $k$, the more simulations are needed achieve the overlap.

In [ ]:
# Simulation Setup
biaspot = potentials1D.harmonicOscillatorPotential(k=10, x_shift=5.5)

The `addedPotentials` function of the biasOneD class wraps any two 1D potentials together. Therefore, it is straightforward to generate the enhanced sampling system from the original and biased potential.


In [ ]:
# Add the bias and the original system
totpot = potentials1D.addedPotentials(V, biaspot)

All subsequent steps are identical to the unbiased system. Note, that the starting position of the simulation has to match to the $x_{shift}$ parameter (i.e. should be reasonable close to $x_{shift}$) in order to avoid starting the simulation at very high energy states.

In [ ]:
system2 = system(potential=totpot, sampler=sampler,
                 start_position=4.2,  temperature=temperature)

In [ ]:
# simulate
sim_steps = 2000
cur_state = system2.simulate(sim_steps, withdraw_traj=True, init_system=False)

In [ ]:

print("Trajectory length: ", len(system2.trajectory))
print()
print("last_state: ", cur_state)
print(len(system2.trajectory))
system2.trajectory.head()

In order to visualize the resulting potential we again use the plotting function `simulation_analysis_plot`. The visualization shows that umbrella sampling samples the high energy transition region around x=5.5 very well. In contrast, the unbiased simulation (see above) was stuck at the minimum around x=4.   

In [ ]:
# plot
simulation_analysis_plot(system2, title="Position Langevin", limits_coordinate_space=list(
    range(0, 10)), oneD_limits_potential_system_energy=[0, 30])

_______________________________________________

#### Temperature Replica Exchange / Parallel Tempering
Temperature replica exchange, also called parallel tempering, is an enhanced sampling method that runs multiple copies of the simulated system at different temperatures. After a specific number of simulation steps, the current coordinate is exchanged with a simulation at a different temperature. However, this exchange is only triggered if a certain condition, e.g. the Metropolis criterion, is fulfilled. This approach has the advantage that one can couple high temperature simulations, that cross energy barriers quickly, with lower temperature systems that sample the minima. Therefore, thermodynamic properties can be calculated with higher precision as more local minima can be explored.


In our example we will perform a temperature replica exchange simulation with two systems. The temperatures of these two systems are defined by the $T\_values$ parameter. We first define how many simulation steps we run per replica and how often the system should try to exchange the coordinates ($simulation\_steps\_total\_per\_approach$ and $trials$). We wrap our system with the replica exchange conditions using the $replica\_exchange.temperatureReplicaExchange$ class. 

For our example simulation we use a common metropolis monte carlo criterium for RE approaches exchanging the coordinates of replica i and replica j,
   
   $p_{ij} = min(1, \exp{((H_{i}(r_i)+H_{j}(r_j))-(H_{i}(r_j)+H_{j}(r_i)))(\frac{1}{k_B T_i}-\frac{1}{k_B T_j})})$

Note that every second trial is not accepted, as the algorithm alternates the partner of the pairwise exchange. Therefore every second exchange is a border exchange, which is in this case of two replicas not exchanging at all.
Subsequently, the simulation is performed as in our previous examples. 

In [ ]:
# Ensemble Settings:
T_values = np.array([10, 1])
simulation_steps_total_per_approach = 2000
trials = 10
steps_between_trials = simulation_steps_total_per_approach//trials

print("DO trials: ", trials, "steps: ", steps_between_trials)

In [ ]:
# Simulation Setup
system_replica = system(potential=V, sampler=sampler,
                        start_position=4.2,  temperature=temperature)

In [ ]:
# define the replica exchange criteria
ensemble = replica_exchange.temperatureReplicaExchange(system=system_replica, temperature_range=T_values,
                                                       exchange_criterium=None, steps_between_trials=steps_between_trials)

In [ ]:
# simulate
cur_state = ensemble.simulate(trials, reset_ensemble=True)
replica_trajs = ensemble.replica_trajectories

We now visualize which part of the potential energy surface our simulations sampled. For that reason, we color-code  by the temperature of the replica.
We start both replicas in the global minimum at $r=4$. One replica is at the high temperature (T=4, red), one at the low temperature (T=0.1, blue). The replica at low temperature intensely samples this minimum, whereas the replica at high temperature is able to overcome the energy barriers surrounding the minimum. Therefore, only states from the high temperature replica are observed at the energy barriers.

After 200 steps the coordinates between our two simulations are exchanged. If the high temperature replica crossed the energy barrier before, the low temperature replica inherits these coordinates and starts in a different minimum. This minimum is then intensely sampled. Using multiple exchange trials, the low temperature replicate can sample all four minima without the need to cross the energy barriers in the low temperature replica.

In [ ]:
# plot
keys = sorted(list(replica_trajs.keys()), reverse=False)
positions = np.linspace(0, 10)
fig, ax = plt.subplots(ncols=1)


# plot potential energy surface
ax.plot(positions, ensemble.replicas[0].potential.ene(
    positions), c="grey", alpha=0.7, zorder=-60)


colormap = {0: 'red', 1: 'blue'}

for traj in keys:
    T = round(ensemble.replicas[traj].temperature, 2)
    # min_e = np.min(replica_trajs[traj].total_potential_energy[eqil:])
    ax.scatter(replica_trajs[traj].position, replica_trajs[traj].total_potential_energy,
               label="replica T="+str(T), c=colormap[traj], alpha=0.5)

    ax.set_ylim([0, 20])
    ax.set_xlim([0, 10])
    ax.set_xlabel("r")
    ax.set_ylabel("$V/[kT]$")
    ax.legend()
    ax.set_title("Temperature replica exchange")
    fig.show()
# Time-dependent bias

_________________________

### Exercise

Please play with simulation length and temperatures of replicas: T_values

What are your observations?

_______________________________________

### Time-dependent bias

Time-dependent biasing methods update the bias during the simulation time. A frequently used time-dependent method is metadynamics/local elevation. There, a gaussian potential is added to the positions that were already visited during the simulation. Therefore, visiting these positions again, is energetically less favorable then in the previous visit (energetic penal).

Note, that in case of a Gaussian bias potential, the mean of the Gaussian is set to the current position of the particle and its width should be chosen small enough to avoid a big penal for neighboring states. Step by step the energetic minima are "flooded" and the particle can cross barriers more easily. In most applications the bias is only added every $n^{th}$ step to iterate between free diffusion and biasing steps.

#### Metadynamics / Local elevation

We first define the original four-well potential. To apply metadynamics/local elevation we use the `metadynamicsPotential` function. In the initialization we have to define the height ($amplitude$) and width ($sigma$) of the gaussian bias function added. This bias potential is added every $n\_trigger$ steps to the current position.

Adding more and more potentials every step leads to an energy function that demands more and more computation time every step. To avoid slowdown of the simulation the metadynamic bias is usually stored and calculated grid based. This allow a much faster simulation but comes at the cost of additional input parameter and small numerical errors in the bias force calculations. To initialize the grid, the user has to define the minimum x-position ($bias\_grid\_min$) and the maximum x-position ($bias\_grid\_max$) the grid should cover as well as the number of grid bins. Note, that no bias will be added to values below $bias\_grid\_min$ or above $bias\_grid\_max$.

In our example system we want to sample all four energy minima. Therefore, it is sufficient to set the grid between 0 and 10, which covers all four minima.

For the metadynamics/local elevation simulation we will reduce the simulation length by the factor of 10. This makes it easier to visually distinguish, and thus understand, the effect of metadynamics/local elevation.

In [ ]:
sim_steps = 200  # reduced simulation length

# Simulation Setup
# Performe metadynamics
totpot = potentials1D.metadynamicsPotential(V, amplitude=.3, sigma=.21, n_trigger=10,
                                            bias_grid_min=0, bias_grid_max=10, numbins=1000)

system4 = system(potential=totpot, sampler=sampler,
                 start_position=4,  temperature=1)

# simulate
cur_state = system4.simulate(sim_steps, withdraw_traj=True, init_system=False)

print("Trajectory length: ", len(system4.trajectory))
print()
print("last_state: ", cur_state)
print(len(system4.trajectory))
system4.trajectory.head()

Metadynamic systems are visualized with the default `simulation_analysis_plot` function. `simulation_analysis_plot` will display the resulting potential after the last simulation step.

In [ ]:
# plot
simulation_analysis_plot(system4, title="position Langevin",
                         limits_coordinate_space=list(range(0, 10)))

In the first panel of the visualization we can see how the system's energy minimum around x=4 was flooded and the particle can cross neighboring energy barriers more easily. The longer one simulates, the flatter the whole energy surface become. Note however, that artifacts can arise once the system leaves the grid defined by $bias\_grid\_min$ and $bias\_grid\_max$. Therefore, these parameters have to be selected very carefully.

________________________________________________

### Exercise

Please play with simulation length and the metadynamics parameters: amplitude, sigma, n_trigger

What do you observe?